In [1]:
import os

import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import numpy as np
from scipy import stats

from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.linear_model import LinearRegression

import ProjectLibrary as pl

Setting up hyperparameters version - model version, testing - debug or output. 
run_All - run a small and big dataset. Otherwise just small

In [2]:
version = 3
testing = 0
run_ALL = 0

Loading Data

In [3]:
path = "DataSets/"
DataSet_A = pd.read_csv(path + 'Study_A.csv').iloc[:,0:39]
DataSet_B = pd.read_csv(path + 'Study_B.csv').iloc[:,0:39]
DataSet_C = pd.read_csv(path + 'Study_C.csv').iloc[:,0:39]
DataSet_D = pd.read_csv(path + 'Study_D.csv').iloc[:,0:39]
DataSet_E = pd.read_csv(path + 'Study_E.csv').iloc[:,0:39]

Sample_Sub = pd.read_csv(path + "sample_submission_PANSS.csv")

DataSet_All = pd.concat([DataSet_A, DataSet_B, DataSet_C, DataSet_D, DataSet_E]) if run_ALL else None

Cleaning the DataSet of all the repeated mesurments, takes long

In [4]:
DataSet_E_M = pl.CleanRepeates(DataSet_E)

Done    10.19%
Done    20.39%
Done    30.58%
Done    40.77%
Done    50.97%
Done    61.16%
Done    71.36%
Done    81.55%
Done    91.74%


Adds Visit Time column to a DataSet

In [5]:
DataSet_E_M = pl.addVisitTime(DataSet_E_M)
DataSet_All_M = pl.addVisitTime(DataSet_All)  if run_ALL else None

Drops unrequired columns "TxGroup", "Study" from a DataSet

In [6]:
DataSet_E_M = DataSet_E_M.drop(columns=["TxGroup", "Study"])
DataSet_All_M = DataSet_All_M.drop(columns=["TxGroup", "Study"])  if run_ALL else None

Encodes as dummies the country

In [7]:
DataSet_E_M = pd.get_dummies(DataSet_E_M, columns=["Country"])
DataSet_All_M = pd.get_dummies(DataSet_All_M, columns=["Country"])  if run_ALL else None

Adds weighted avarage speed of change column to a DataSet

In [8]:
DataSet_E_M = AddSmartChange(DataSet_E_M)
DataSet_All_M = AddSmartChange(DataSet_All_M) if run_ALL else None

NameError: name 'AddSmartChange' is not defined

Creating test & train data - our model predicts the day of last visit and the PANSS_Total

In [ ]:
DataSet_E_Test = pd.DataFrame(columns=DataSet_E_M.columns)
DataSet_E_Train = pd.DataFrame(columns=DataSet_E_M.columns)
PatientID = -1
row_n = -1

while row_n + 1 < DataSet_E_M.shape[0]:
    if PatientID != DataSet_E_M.iloc[row_n + 1]["PatientID"]:
        if PatientID != -1:
            DataSet_E_Test = DataSet_E_Test.append(DataSet_E_M.iloc[row_n])
        PatientID = DataSet_E_M.iloc[row_n]["PatientID"]
    else:
        DataSet_E_Train = DataSet_E_Train.append(DataSet_E_M.iloc[row_n])
        
    print("Done %8.2f%%" % ((row_n+1)/DataSet_E.shape[0] * 100)) if not row_n % 100 else None

    row_n += 1
    
DataSet_E_Test = DataSet_E_Test[DataSet_E_Test["VisitDay"] > 0]

DataSet_E_Test = DataSet_E_Test.reset_index()
DataSet_E_Train = DataSet_E_Train.reset_index()

Sel_E_Output = DataSet_E_M[DataSet_E_M["PatientID"].isin(Sample_Sub["PatientID"])].reset_index()
    
print("Finished!")

Clustering the Data for each Patient using day zero scores and SmartChange

In [ ]:
clust_columns = ["Country_Russia", "Country_UK", "Country_USA", "SmartChange", "PANSS_Total"]
number_cluster = 20

mask_TrainE = DataSet_E_Train["VisitDay"] == 0
mask_SelectE = Sel_E_Output["VisitDay"] == 0

kmeans_TrainE = KMeans(n_clusters=number_cluster, random_state=0).fit(NormalDataFrame(DataSet_E_Train.loc[mask_TrainE, clust_columns]))
kmeans_SelectE = KMeans(n_clusters=number_cluster, random_state=0).fit(NormalDataFrame(Sel_E_Output.loc[mask_SelectE, clust_columns]))

Full_Clust_DataSet_E_Train = [DataSet_E_Train["PatientID"][nrow] for nrow in range(DataSet_E_Train.shape[0])]
Full_Clust_DataSet_E_Train


Creating All XGB permutations for training - Work in progress...

In [ ]:
# mask_TrainE = DataSet_E_Train["VisitDay"] == 0

# for cluster in range(number_cluster):
#     clust_PatientID = DataSet_E_Train["PatientID"][mask_SelectE][kmeans_TrainE.labels_ == cluster].unique()
#     XGB_DATAFRAME = DataSet_E_Train[][DataSet_E_Train["PatientID"].isin(clust_PatientID)]